In [9]:
import pandas as pd

df= pd.read_csv('data/test/testnya.csv')
df2= pd.read_csv('data/test/multi_log_reg/multi_nya.csv')

In [10]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,rev_dist
0,0,0,0,^NYA,2008-08-01 00:00:00-04:00,-59.559570,0,0.0,0.0,0.00,0.00000,0.00000,0.000000
1,1,12,12,^NYA,2008-08-04 00:00:00-04:00,-110.509766,0,0.0,0.0,0.00,0.00000,0.00000,0.000000
2,2,24,24,^NYA,2008-08-05 00:00:00-04:00,203.149414,2,12.3,6.3,6.15,11869.40281,15464.78374,0.000084
3,3,36,36,^NYA,2008-08-06 00:00:00-04:00,29.600586,0,0.0,0.0,0.00,0.00000,0.00000,0.000000
4,4,48,48,^NYA,2008-08-07 00:00:00-04:00,-163.299805,0,0.0,0.0,0.00,0.00000,0.00000,0.000000


In [13]:
df2 = df2.rename(columns={'Date': 'date'})
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
#df2['date'] = pd.to_datetime(df2['date']).dt.tz_localize('America/New_York')
df2['date'] = pd.to_datetime(df2['date']).dt.strftime('%Y-%m-%dT%H:%M:%S%z')
df2_subset = df2[['date', 'effect']]

# Merge df and df2_subset on the Date column
merged_df = pd.merge(df, df2_subset, on='date', how='left')
merged_df.head()


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,rev_dist,effect
0,0,0,0,^NYA,2008-08-01T00:00:00-0400,-59.559570,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
1,1,12,12,^NYA,2008-08-04T00:00:00-0400,-110.509766,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
2,2,24,24,^NYA,2008-08-05T00:00:00-0400,203.149414,2,12.3,6.3,6.15,11869.40281,15464.78374,0.000084,0.0
3,3,36,36,^NYA,2008-08-06T00:00:00-0400,29.600586,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
4,4,48,48,^NYA,2008-08-07T00:00:00-0400,-163.299805,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,1.0


In [14]:
merged_df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1','Unnamed: 0'], inplace=True)
merged_df.head()

,Ticker,date,change,num,sum,max,avg,min_dist,max_dist,rev_dist,effect
0,^NYA,2008-08-01T00:00:00-0400,-59.559570,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
1,^NYA,2008-08-04T00:00:00-0400,-110.509766,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
2,^NYA,2008-08-05T00:00:00-0400,203.149414,2,12.3,6.3,6.15,11869.40281,15464.78374,0.000084,0.0
3,^NYA,2008-08-06T00:00:00-0400,29.600586,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,0.0
4,^NYA,2008-08-07T00:00:00-0400,-163.299805,0,0.0,0.0,0.00,0.00000,0.00000,0.000000,1.0


In [15]:
merged_df['effect'].unique()

array([ 0.,  1., nan,  3.,  2.])

In [16]:
import statsmodels.api as sm

# Add a constant to the independent variables (for the intercept term)
X = sm.add_constant(merged_df[['num', 'max', 'min_dist']])
y = merged_df['effect']

# Fit the logistic regression model
mlogit_model = sm.MNLogit(y, X)
result = mlogit_model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.292200
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:                 effect   No. Observations:                 3773
Model:                        MNLogit   Df Residuals:                     3761
Method:                           MLE   Df Model:                            9
Date:                Sat, 22 Feb 2025   Pseudo R-squ.:                  0.1007
Time:                        12:46:05   Log-Likelihood:                -1102.5
converged:                       True   LL-Null:                       -1226.0
Covariance Type:            nonrobust   LLR p-value:                 4.313e-48
  effect=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4127      0.075    -32.345      0.000      -2.559      -2.266
num           -0.2506      0.